# Project: NIFA

Cassandra Wattenburger, 09/26/2022

### Notes:
* QIIME2 v2021.4

# Pipeline information

### Pipeline to process raw sequences with DADA2 ###
* Prep for import into QIIME2 (modify sequence IDs and combine two index files)
* Import into QIIME2
* Demultiplex
* Denoise and merge with DADA2
* Prepare ASV tables and representative sequences *(Note: sample names starting with a digit will break this step)*
* Classify sequences
* Construct phylogenetic tree
* Export from QIIME2

*100% Appropriated from the "Atacama Desert Tutorial" for QIIME2*

### Pipeline can handle both 16S rRNA gene and ITS sequences ###
* Tested on 515f and 806r
* Tested on ITS1

# Before you start

### Commands to install dependencies ####
##### || QIIME2 and biopython ||
QIIME2 is still actively in development with frequent new releases. Check for the most up-to-date version and use that.

Install QIIME2: 
* <https://docs.qiime2.org/2017.11/install/native/#install-qiime-2-within-a-conda-environment>, follow the instructions to install QIIME2 in Linux (64-bit).

Activate the QIIME2 environment:
* source activate [qiime2-pipeline-name]

After you install and activate the QIIME2 environment, you must also install biopython for the barcode concatenation step to work. To install biopython make sure the QIIME2 environment is activated and run:
* conda install -c anaconda biopython

Install cutadapt to the QIIME2 environment as well. Cutadapt removes primers from the sequences.
* conda install -c bioconda cutadapt

##### || Copyrighter rrn database ||
The script will automatically install the curated GreenGenes rrn attribute database: https://github.com/fangly/AmpliCopyrighter

#### Citations
* Caporaso, J. G., Kuczynski, J., Stombaugh, J., Bittinger, K., Bushman, F. D., Costello, E. K., *et al.* (2010). QIIME allows analysis of high-throughput community sequencing data. Nature methods, 7(5), 335-336.

* Angly, F. E., Dennis, P. G., Skarshewski, A., Vanwonterghem, I., Hugenholtz, P., & Tyson, G. W. (2014). CopyRighter: a rapid tool for improving the accuracy of microbial community profiles through lineage-specific gene copy number correction. Microbiome, 2(1), 11.

### Using jupyter notebook screens ###

With the QIIME2 environment activated, open your jupyter notebook screen in the directory containing this script:
* jupyter-n [screen-name] [port #]

See [these instructions](https://github.com/buckleylab/Buckley_lab_protocols/blob/master/Using_the_server/getting_started_on_server.md#make-jupyter-notebook-screens-command) for how to set up and use this command on the server.

### Directory and data organization ###

This pipeline assumes that you've organized your data in a certain way:
* Each library of raw data is contained in a separate directory
* Each library has a separate working directory within a larger project directory
* Tree construction assumes all 16S libraries processed together will be analyzed together, so one tree is made based on all 16S libraries and is placed in a separate tree directory within the project directory

### Troubleshooting tip ###
Replace os.system with print, and copy/paste the output into the command line to view the error message.

# Step 1: User Input

Metadata requirements:
* Must be located in each library directory
* Must be .tsv format 
* First column is named "SampleID" with sample names as rows
* Contains another column named "BarcodeSequence" with the relevant barcode seqeunces as rows (rev. comp. reverse barcode sequence concatenated with forward barcode sequence)

In [2]:
import os, re, numpy as np

# Prepare an object with the name of the library and all related file paths
# datasets = [['library name prefix', 'processed data directory path', 'raw data directory', 'read1 file name', 'read2 file name', 'index1 file name', 'index2 file name', 'metadata file name', 'domain of life (bacteria or fungi)'], ...]
project = "/home/cassi/NIFA/data_amplicon" # this can be the same as the library directory if you only have one library to process
libraries = [['NIFA', 
             '/home/cassi/NIFA/data_amplicon/NIFA', 
             '/home/backup_files/raw_reads/NIFA.Cassi.2022',
             '170932_KLGRY_NIFA_S1_R1_001.fastq.gz', 
             '170932_KLGRY_NIFA_S1_R2_001.fastq.gz', 
             '170932_KLGRY_NIFA_S1_I1_001.fastq.gz', 
             '170932_KLGRY_NIFA_S1_I2_001.fastq.gz', 
             'NIFA_demultiplex.tsv',
             'bacteria']]

# Set # of processors
processors = 10

# Which bacterial database will you use? Silva or GreenGenes
db = "Silva"

# Phylogenetic tree (non-fungal data)
treename = "NIFA" # name prefix for the tree file

## Enter minimum support for keeping QIIME classification
# Note: Classifications that do not meet this criteria will be retained, labeled 'putative'
min_support = 0.9

# Step 2: Truncate sequence identifiers

This step removes a portion at the end of the sequence ID that is incompatible with QIIME2. It will also create a modified directory in your raw data directory to house the modified data. The original raw data will not be modified, only the copies.

Slow step.

In [2]:
for library in libraries:
    raw = library[2]
    read1 = library[3]
    read2 = library[4]
    index1 = library[5]
    index2 = library[6]
    
    # Create a directory to place modified sequence data
    if not os.path.isdir(os.path.join(raw, "modified")):
        !mkdir $raw/modified
    
    # Copy/paste all raw sequence data into modified directory
    !cp $raw/*.fastq.gz $raw/modified
    
    # Decompress all files
    !unpigz $raw/modified/*.fastq.gz
    
    # Decompressed file names
    read1decomp = re.sub(".fastq.gz", ".fastq", read1)
    read2decomp = re.sub(".fastq.gz", ".fastq", read2)
    index1decomp = re.sub(".fastq.gz", ".fastq", index1)
    index2decomp = re.sub(".fastq.gz", ".fastq", index2)
    
    # Remove problematic part of sequence IDs
    !sed 's/\ [0-9]:[YN]:[0-9]:[0-9]$//g' $raw/modified/$read1decomp > $raw/modified/read1_mod.fastq
    !sed 's/\ [0-9]:[YN]:[0-9]:[0-9]$//g' $raw/modified/$read2decomp > $raw/modified/read2_mod.fastq
    !sed 's/\ [0-9]:[YN]:[0-9]:[0-9]$//g' $raw/modified/$index1decomp > $raw/modified/index1_mod.fastq
    !sed 's/\ [0-9]:[YN]:[0-9]:[0-9]$//g' $raw/modified/$index2decomp > $raw/modified/index2_mod.fastq
    
    # Delete file copies
    !rm $raw/modified/$read1decomp
    !rm $raw/modified/$read2decomp
    !rm $raw/modified/$index1decomp
    !rm $raw/modified/$index2decomp

# Step 3: Remove primer from sequences

There may still be portions of the primers left in the read sequences that need to be removed. Use cutadapt to remove those portions. Read1 will have the reverse complement of the reverse primer and read2 will have the reverse complement of the forward primer in some sequences on the 3' end.

You will get a warning that the adapter is preceded by "A" or "G" extremely often. These are the link sequences in the primer.

In [3]:
for library in libraries:
    raw = library[2]
    read1 = library[3]
    read2 = library[4]
    
    !cutadapt -a ATTAGAWACCCBDGTAGTCC -o $raw/modified/read1_noprimer.fastq $raw/modified/read1_mod.fastq
    !cutadapt -a TTACCGCGGCKGCTGGCAC -o $raw/modified/read2_noprimer.fastq $raw/modified/read2_mod.fastq
    
    # Delete unneeded intermediate files
    !rm $raw/modified/read*_mod.fastq

This is cutadapt 3.4 with Python 3.8.8
Command line parameters: -a ATTAGAWACCCBDGTAGTCC -o /home/backup_files/raw_reads/NIFA.Cassi.2022/modified/read1_noprimer.fastq /home/backup_files/raw_reads/NIFA.Cassi.2022/modified/read1_mod.fastq
Processing reads on 1 core in single-end mode ...
[--=8        ] 00:02:54    17,197,051 reads  @     10.1 µs/read;   5.93 M reads/minute
Finished in 174.03 s (10 µs/read; 5.93 M reads/minute).

=== Summary ===

Total reads processed:              17,197,051
Reads with adapters:                    92,105 (0.5%)
Reads written (passing filters):    17,197,051 (100.0%)

Total basepairs processed: 4,316,459,801 bp
Total written (filtered):  4,314,935,691 bp (100.0%)

=== Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 92105 times

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20 bp: 2

Bases preceding removed adapters:
  A: 19.1%
  C: 25.0%
  G: 36.9%
  T: 19.0%
  none/other: 0.0%

Overview of removed sequences
length	co

# Step 4: Filter short reads

Remove sequences with from primer-removed data with less than 100 bp from all files (too short).

SLOW STEP.

In [ ]:
for library in libraries:
    raw = library[2]
    os.system(' '.join([
        "python /home/cassi/scripts/qiime2_filter_shortreads.py",
        raw+"/modified/"
    ]))
    
    # Remove unneeded intermediate files
    !rm $raw/modified/*_noprimer.fastq
    !rm $raw/modified/index*_mod.fastq
    
    # Recompress modified read files
    !pigz $raw/modified/read*_filtered.fastq

# Step 5: Concatenate barcodes

This step calls a custom script, "concatenate_barcodes_qiime2.py". The script must be shuttled to the command line instead of run directly in jupyter notebooks because jupyter has memory issues that truncates the barcodes file without an error (I think).

This script requires your index1 and index2 files to be named index1_mod.fastq and index2_mod.fastq and be located in a directory within the raw read directory called 'modified'. This should have been taken care of in earlier steps in this pipeline.

In [5]:
for library in libraries:
    raw = library[2]
    
    os.system(' '.join([
        "python /home/cassi/scripts/qiime2_concatenate_barcodes.py",
        raw+"/modified"]))
  
    # Recompress modified index files and newly created barcodes.fastq file
    !pigz $raw/modified/*.fastq   

# Step 6: Move raw data to library directories

Creates intermediate directory in library directory. All subsequent files except for the final files will be placed there.

In [6]:
for library in libraries:
    proc = library[1]
    raw = library[2]
    
    # Create output directory if it doesn't exist already
    if not os.path.isdir(os.path.join(proc, "intermediate")):
        !mkdir $proc/intermediate
    
    # Create a symbolic link to the read data
    # QIIME2 import requires a directory containing files named: forward.fastq.gz, reverse.fastq.gz and barcodes.fastq.gz 
    !ln -s $raw/modified/read1_filtered.fastq.gz $proc/intermediate/forward.fastq.gz
    !ln -s $raw/modified/read2_filtered.fastq.gz $proc/intermediate/reverse.fastq.gz
    
    # Move concatenated barcodes to project directory
    !cp $raw/modified/barcodes.fastq.gz $proc/intermediate/

# Step 7: Import into QIIME2

In [7]:
for library in libraries:
    name = library[0]
    proc = library[1]
    
    os.system(' '.join([
        "qiime tools import",
        "--type EMPPairedEndSequences",
        "--input-path "+proc+"/intermediate",
        "--output-path "+proc+"/intermediate/"+name+".qza"
    ]))

# Step 8: Demultiplex

The barcode you supply to QIIME is now a concatenation of your forward and reverse barcode. Your 'forward' barcode is actually the reverse complement of your reverse barcode and the 'reverse' is your forward barcode.

Slow step.

In [ ]:
for library in libraries:
    name = library[0]
    proc = library[1]
    metadata = library[7]
    
    os.system(' '.join([
        "qiime demux emp-paired",
        "--m-barcodes-file "+proc+"/"+metadata,
        "--m-barcodes-column BarcodeSequence",
        "--p-no-golay-error-correction",
        "--i-seqs "+proc+"/intermediate/"+name+".qza",
        "--o-per-sample-sequences "+proc+"/intermediate/"+name+".demux",
        "--o-error-correction-details "+proc+"/intermediate/"+name+".demux-details.qza"
    ]))

# Step 9: Visualize quality scores

Drop output from below command into https://view.qiime2.org

In [16]:
for library in libraries:
    name = library[0]
    proc = library[1]
    
    os.system(' '.join([
        "qiime demux summarize",
        "--i-data "+proc+"/intermediate/"+name+".demux.qza",
        "--o-visualization "+proc+"/intermediate/"+name+".demux.qzv"
    ]))

# Step 10: Trimming parameters | USER INPUT REQUIRED

Based on the quality scores of the bp along the reads, choose trim and truncate values for the forward and reverse reads. Trim refers to the start of a sequence and truncate the total length (i.e. number of bases to remove from end).

All trimming parameters must be the same for datasets that will be directly compared to one-another because ASVs are determined, in part, by sequence length.

In [19]:
## Input your trimming parameters into a python dictionary for all libraries
# trim_dict = {"LibraryName1":[trim_forward, truncate_forward, trim_reverse, truncate_reverse],
#              "LibraryName2":[trim_forward, truncate_forward, trim_reverse, truncate_reverse],
#               etc...}

# The example in the Atacama Desert Tutorial trims 13 bp from the start of each read and does not remove any bases from the end of the 150 bp reads:
#  --p-trim-left-f 13 \  
#  --p-trim-left-r 13 \
#  --p-trunc-len-f 150 \
#  --p-trunc-len-r 150

trim_dict = {"NIFA":[15,220,15,220]} 

# Step 11: Trim, denoise and join (aka 'merge') reads using DADA2

See the [QIIME2 dada2 denoise-paired documentation](https://docs.qiime2.org/2021.8/plugins/available/dada2/denoise-paired/) for the default parameters used.

Slow step.

In [20]:
for library in libraries:
    name = library[0]
    proc = library[1]
    
    os.system(' '.join([
        "qiime dada2 denoise-paired",
        "--i-demultiplexed-seqs "+proc+"/intermediate/"+name+".demux.qza",
        "--o-table "+proc+"/intermediate/"+name+".table.qza",
        "--o-representative-sequences "+proc+"/intermediate/"+name+".rep-seqs.qza",
        "--o-denoising-stats "+proc+"/intermediate/"+name+".denoising-stats.qza",
        "--p-trim-left-f "+str(trim_dict[name][0]),
        "--p-trim-left-r "+str(trim_dict[name][2]),
        "--p-trunc-len-f "+str(trim_dict[name][1]),
        "--p-trunc-len-r "+str(trim_dict[name][3]),
        "--p-n-threads",
        str(processors)
    ]))

In [7]:
for library in libraries:
    name = library[0]
    proc = library[1]
    
    os.system(' '.join([
        "qiime metadata tabulate",
        "--m-input-file "+proc+"/NIFA/intermediate/"+name+".denoising-stats.qza",
        "--o-visualization "+proc+"/NIFA/intermediate/"+name+".denoising-stats.qzv"
    ]))

# Step 12: Create summary of ASVs

Drop outputs from below command into https://view.qiime2.org

In [21]:
for library in libraries:
    name = library[0]
    proc = library[1]
    metadata = library[7]
    
    os.system(' '.join([
        "qiime feature-table summarize",
        "--i-table "+proc+"/intermediate/"+name+".table.qza",
        "--o-visualization "+proc+"/intermediate/"+name+".table.qzv",
        "--m-sample-metadata-file "+proc+"/"+metadata
    ]))

    os.system(' '.join([
        "qiime feature-table tabulate-seqs",
        "--i-data "+proc+"/intermediate/"+name+".rep-seqs.qza",
        "--o-visualization "+proc+"/intermediate/"+name+".rep-seqs.qzv"
    ])) 

# Step 13: Classify sequences

Different QIIME2 versions can conflict with certain classifier database versions. This section will likely need to be updated. Download the latest classifiers here: https://docs.qiime2.org/2021.4/data-resources/

* Using SILVA v138 pre-built classifier trained on scikit learn 0.24.1.

View output in https://view.qiime2.org

In [ ]:
# Use classifier for chosen database
try:
    if db == "GreenGenes":
        classifier_db = "/home/db/GreenGenes/qiime2_13.8.99_515.806_nb.classifier.qza" # out of date
    else:
        classifier_db = "~/databases/silva/silva-138-99-515-806-nb-classifier.qza"
except:
        classifier_db = "~/databases/silva/silva-138-99-515-806-nb-classifier.qza"
        
for library in libraries:
    name = library[0]
    proc = library[1]
    metadata = library[7]
    domain = library[8]

    # Classify
    if domain == 'bacteria':
        os.system(' '.join([
            "qiime feature-classifier classify-sklearn",
            "--i-classifier",
            classifier_db,
            "--i-reads "+proc+"/intermediate/"+name+".rep-seqs.qza",
            "--o-classification "+proc+"/intermediate/"+name+".taxonomy.qza",
            "--p-n-jobs",
            str(processors)
        ]))

    if domain == 'fungi':
        os.system(' '.join([
            "qiime feature-classifier classify-sklearn",
            "--i-classifier /home/db/UNITE/qiime2_unite_ver7.99_20.11.2016_classifier.qza", # out of date
            "--i-reads "+proc+"/intermediate/"+name+".rep-seqs.qza",
            "--o-classification "+proc+"/intermediate/"+name+".taxonomy.qza",
            "--p-n-jobs",
            str(processors)
        ]))

    # Output summary
    os.system(' '.join([
        "qiime metadata tabulate",
        "--m-input-file "+proc+"/intermediate/"+name+".taxonomy.qza",
        "--o-visualization "+proc+"/intermediate/"+name+".taxonomy-summary.qzv"
    ])) 

# Step 14: Combine representative sequences for tree

In [23]:
# Bacteria

# Create representative sequences file
repseqsbac = []

for library in libraries:
    name = library[0]
    proc = library[1]
    domain = library[8]
    
    # Create list of rep seq files
    if domain == "bacteria":
        repseqsbac.append("--i-data " + os.path.join(proc, "intermediate", name+".rep-seqs.qza"))

# Create tree directory
if not os.path.isdir(os.path.join(project, "tree")):
    !mkdir $project/tree
    
# Merge rep sequences from all bacterial libraries for tree
os.system(' '.join([
    "qiime feature-table merge-seqs",
    " ".join(repseqsbac),
    "--o-merged-data "+project+"/tree/"+treename+".rep-seqs-merged.qza"
]))

# Fungi
for library in libraries:
    name = library[0]
    proc = library[1]
    domain = library[8]
    
    # Create representative sequences file
    repseqsfung = []
    
    if domain == "fungi":
        repseqsfung.append("--i-data " + os.path.join(proc, "intermediate", name+".rep-seqs.qza"))

# Step 15: Make phylogenetic tree

In [24]:
for library in libraries:
    domain = library[8]

    if domain == "bacteria":
        # Generate alignment with MAFFT
        os.system(' '.join([
            "qiime alignment mafft",
            "--i-sequences "+project+"/tree/"+treename+".rep-seqs-merged.qza",
            "--o-alignment "+project+"/tree/"+treename+".rep-seqs-merged-aligned.qza",
            "--p-n-threads",
            str(processors)
        ]))

        # Mask hypervariable regions in alignment
        os.system(' '.join([
            "qiime alignment mask",
            "--i-alignment "+project+"/tree/"+treename+".rep-seqs-merged-aligned.qza",
            "--o-masked-alignment "+project+"/tree/"+treename+".rep-seqs-merged-aligned-masked.qza",
        ]))

        # Generate tree with FastTree
        os.system(' '.join([
            "qiime phylogeny fasttree",
            "--i-alignment "+project+"/tree/"+treename+".rep-seqs-merged-aligned-masked.qza",
            "--o-tree "+project+"/tree/"+treename+".tree-unrooted.qza",
            "--p-n-threads",
            str(processors)
        ]))

        # Root the tree
        os.system(' '.join([
            "qiime phylogeny midpoint-root",
            "--i-tree "+project+"/tree/"+treename+".tree-unrooted.qza",
            "--o-rooted-tree "+project+"/tree/"+treename+".tree-rooted.qza"
        ]))

# Step 16: Reformat taxonomy

Define function to tidy the taxonomy and make it compatible with phyloseq.

In [25]:
def format_taxonomy(tax_dirty, min_support):
    output = open(re.sub(".tsv","-fixed.tsv",tax_dirty), "w")
    
    full_rank_length = 7
    output.write("\t".join(["ASV","Domain","Phylum","Class","Order","Family","Genus","Species"])+"\n")

    with open(tax_dirty, "r") as f:
        next(f)

        for line in f:
                line = line.strip()
                line = line.split("\t")

                read_id = line[0]
                tax_string = line[1]
                
                ## Remove taxonomy prefixes and underscores (only coded for Silva classifications so far)
                if db == "Silva":
                    tax_string = re.sub("[a-z]__", "", tax_string)
                    tax_string = re.sub("_", " ", tax_string)

                # Split full rank into ranks
                full_rank = tax_string.split(";")

                ## Identify the lowest classified taxonomic rank
                # Account for cases when a taxonomic rank contains an empty space (common in GreenGenes output)
                last_classified = full_rank[len(full_rank)-1]            

                count = 1
                while last_classified == " ":
                    last_classified = full_rank[len(full_rank)-count]
                    count = count + 1

                # Annotate the last classified as 'putative' if it does not meet the minimum support criteria
                if float(line[2]) < float(min_support):
                        full_rank[full_rank.index(last_classified)] = "putative "+last_classified
                        last_classified = "putative "+last_classified

                # Add in columns containing unclassified taxonomic information
                for n in range(full_rank.index(last_classified)+1, full_rank_length, 1):               
                    try:
                        full_rank[n] = "unclassified "+last_classified
                    except:
                        full_rank.append("unclassified "+last_classified)

                # Write taxonomy to file
                output.write(read_id+"\t"+'\t'.join(full_rank)+"\n")
    return()

# Step 17: Export from QIIME2

All final files will be placed in 'final' directory in library directories. Final tree will be in a 'tree' directory in the project directory.

In [26]:
for library in libraries:
    name = library[0]
    proc = library[1]
    metadata = library[7]

    # Final output paths
    fasta_final = proc+"/final/"+name+".rep-seqs-final.fasta"
    tax_final= proc+"/final/"+name+".taxonomy-final.tsv"
    count_final = proc+"/final/"+name+".counts-final.biom"
    
    # Make final data directories
    if not os.path.isdir(os.path.join(proc, "final")):
        !mkdir $proc/final
        
    # Export ASV table
    os.system(' '.join([
        "qiime tools export",
        "--input-path "+proc+"/intermediate/"+name+".table.qza",
        "--output-path "+proc+"/intermediate/"
    ]))

    # Export taxonomic classifications
    os.system(' '.join([
        "qiime tools export",
        "--input-path "+proc+"/intermediate/"+name+".taxonomy.qza",
        "--output-path "+proc+"/intermediate/"
    ]))
    
    # Reformat classifications  
    format_taxonomy(proc+"/intermediate/taxonomy.tsv", min_support)

    # Export representative sequences
    os.system(' '.join([
        "qiime tools export",
        "--input-path "+proc+"/intermediate/"+name+".rep-seqs.qza",
        "--output-path "+proc+"/intermediate/"
    ]))
    
    # Rename exported files and move to final directory
    !mv $proc/intermediate/dna-sequences.fasta $fasta_final
    !mv $proc/intermediate/feature-table.biom $count_final
    !mv $proc/intermediate/taxonomy-fixed.tsv $tax_final
    
    # Reformat count table
    tmp_tsv = re.sub(name+".counts-final.biom", "tmp.tsv", count_final)
    tmp2_tsv = re.sub(name+".counts-final.biom", "tmp2.tsv", count_final)
    count_tsv = re.sub(".biom", ".tsv", count_final)
    !biom convert -i $count_final -o $tmp_tsv --to-tsv # conver to .tsv
    !tail -n +2 $tmp_tsv > $tmp2_tsv # remove header
    !sed 's/\#OTU ID/ASV/g' $tmp2_tsv > $count_tsv # replace OTU with ASV
    !rm $tmp_tsv
    !rm $tmp2_tsv
    
# Export tree
os.system(' '.join([
    "qiime tools export",
    "--input-path "+project+"/tree/"+treename+".tree-rooted.qza",
    "--output-path "+project+"/tree/"
]))

# Rename tree file
tree_final = project+"/tree/"+treename+".tree-final.nwk"
!mv $project/tree/"tree.nwk" $tree_final

In [27]:
# Export tree
os.system(' '.join([
    "qiime tools export",
    "--input-path "+project+"/tree/"+treename+".tree-rooted.qza",
    "--output-path "+project+"/tree/"
]))

# Rename tree file
tree_final = project+"/tree/"+treename+".tree-final.nwk"
!mv $project/tree/"tree.nwk" $tree_final

In [29]:
# Export merged rep seqs
os.system(' '.join([
        "qiime tools export",
        "--input-path "+project+"/tree/"+treename+".rep-seqs-merged.qza",
        "--output-path "+project+"/tree/"
    ]))

qiime tools export --input-path /home/cassi/NIFA/data_amplicon/tree/NIFA.rep-seqs-merged.qza --output-path /home/cassi/NIFA/data_amplicon/tree/


# Done!